## Check / Prepare data for algae blooms identification.


The idea is to take the modeled data and train a machine learning (ML) model on that data, then try to use on the observational data.
The reason - models can't predict very well the exact time and location of algae blooms but they reproduce the physics/biogeochemistry of it.
Thus, the intuition to check is that a ML model trained on modelled data will be able to predict blooms on observational data.

In [ ]:
import glob
from pathlib import Path

import numpy as np  # noqa: F401
import pandas as pd  # noqa: F401
import xarray as xr
import matplotlib.pyplot as plt  # noqa: F401

from blooms_ml.utils import (
    sample_stations,
    extract_stations_rho,
    extract_stations_u,
    extract_stations_v,
    merge_edges_to_centers,
    append_rho_profiles,
    normalize_series,
    plot_variable,
    plot_confusion_matrix,
)

There is the output of hydrophysical+biogeochemical model of the Hardangerfjord at HPC FRAM.
The files are very huge to download, so I have just mounted a data folder to use them.
This is based on the ROMS hydrophysical and NERSEM biogeochemical models.
Diagnostic files have data about PAR (photosynthetically active radiation).
'Average' files have the rest of the variables.

In [ ]:
files_dia = sorted(glob.glob(f"{Path.home()}/fram_shmiak/ROHO800_hindcast_2007_2019_v2bu/roho800_v2bu_dia/*dia*.nc"))
files_avg = sorted(glob.glob(f"{Path.home()}/fram_shmiak/ROHO800_hindcast_2007_2019_v2bu/roho800_v2bu_avg/*avg*.nc"))
last_n_files = 10

In [ ]:
ds_dia = xr.open_mfdataset(files_dia[-last_n_files:])
ds_avg = xr.open_mfdataset(files_avg[-last_n_files:])

In [ ]:
stations, st_labels, xis, etas = sample_stations(ds_dia, 100)

In [ ]:
p = ds_dia.mask_rho.isel(ocean_time=-1).plot(
    x="xi_rho", y="eta_rho", figsize=(14, 7), cmap='GnBu'
    )
p.axes.scatter(x=xis, y=etas, color='red')
for i, label in enumerate(st_labels):
    p.axes.annotate(label, (xis[i], etas[i]), color='red')

Extract light from the chosen points.

In [ ]:
ds = extract_stations_rho(ds_dia, xis, etas)
ds = merge_edges_to_centers(ds)
df_dia = ds[['light_PAR0', 'P1_netPI']].to_dataframe()

Extract other variables.
There are too many variables, let's take only some of them.

In [ ]:
vars = ['lat_rho', 'lon_rho', 'ocean_time', 's_rho',
        'TotChl', 'P1_c',
        'swradWm2',
        'rho', 'temp', 'salt', 'AKv', 'u', 'v', 'w',
        'N1_p', 'N3_n', 'N5_s', 'O2_o']

In [ ]:
ds_rho = extract_stations_rho(ds_avg, xis, etas)
ds_rho = ds_rho.drop_dims(['eta_u', 'eta_v', 'eta_psi', 'xi_u', 'xi_v', 'xi_psi' ])
ds_u = extract_stations_u(ds_avg, xis, etas)
ds_u = ds_u.drop_dims(['eta_rho', 'eta_v', 'eta_psi', 'xi_rho', 'xi_v', 'xi_psi' ])
ds_v = extract_stations_v(ds_avg, xis, etas)
ds_v = ds_v.drop_dims(['eta_rho', 'eta_u', 'eta_psi', 'xi_rho', 'xi_u', 'xi_psi' ])
ds = xr.merge([ds_rho, ds_u, ds_v])

In [ ]:
ds = merge_edges_to_centers(ds)
ds_subset = ds.drop_vars([var for var in ds.variables if var not in vars])
df = ds_subset.to_dataframe()

In [ ]:
df['light_PAR0'] = df_dia['light_PAR0']
df['P1_netPI'] = df_dia['P1_netPI']

In [ ]:
df

Visualization

In [ ]:
df_station = df.loc[df.index.get_level_values('station') == 3]
df_station = df_station.reset_index()

In [ ]:
df_station

In [ ]:
plot_variable(df_station.pivot(index='s_rho', columns='ocean_time', values='light_PAR0').iloc[::-1])

In [ ]:
plot_variable(df_station.pivot(index='s_rho', columns='ocean_time', values='P1_c').iloc[::-1])

In [ ]:
plot_variable(df_station.pivot(index='s_rho', columns='ocean_time', values='P1_netPI').iloc[::-1])

In [ ]:
plot_variable(df_station.pivot(index='s_rho', columns='ocean_time', values='w').iloc[::-1])

In [ ]:
plot_variable(df_station.pivot(index='s_rho', columns='ocean_time', values='rho').iloc[::-1])

In [ ]:
plot_variable(df_station.pivot(index='s_rho', columns='ocean_time', values='N1_p').iloc[::-1])

In [ ]:
plot_variable(df_station.pivot(index='s_rho', columns='ocean_time', values='N3_n').iloc[::-1])

Data preprocessing.
Using equation of state it is possible to recover density form temperature and salinity.
Extract and append rho profiles.

In [ ]:
df_input = df.drop(columns=['lon_rho', 'lat_rho', 'temp', 'salt', 'u', 'v', 'O2_o', 'AKv'])
df_input = append_rho_profiles(df_input)

In [ ]:
# no light, no blooms, remove dark points, it will help with scaling and sampling
df_input = df_input[df_input['light_PAR0'] > 10]
df_input = df_input.reset_index(drop=True)
df_input

In [ ]:
df_input['label'] = (df_input['P1_netPI'] > 18).astype(np.float32)
df_input

Scaling

In [ ]:
df_input = df_input.drop(columns=['ocean_time', 's_rho'])
df_input

In [ ]:
df_input.iloc[:, :8] = df_input.iloc[:, :8].apply(normalize_series, axis=0)

In [ ]:
df_input['light_PAR0'].plot()

In [ ]:
(df_input['label'] == 1).sum()

In [ ]:
df_input

Features correlation

In [ ]:
df = df_input.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, -1]].sample(frac=1)  # exclude rho profiles
df_bloom = df.loc[df['label'] == 1]
df_clean = df.loc[df['label'] == 0][:(df_input['label'] == 1).sum()]
normal_distributed_df = pd.concat([df_bloom, df_clean])
new_df = normal_distributed_df.sample(frac=1, random_state=42)

In [ ]:
import seaborn as sns

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

corr = df.corr()
sns.heatmap(corr, cmap='coolwarm_r', annot_kws={'size':10}, ax=ax1)
ax1.set_title("Imbalanced Correlation Matrix", fontsize=14)

sub_sample_corr = new_df.corr()
sns.heatmap(sub_sample_corr, cmap='coolwarm_r', annot_kws={'size':10}, ax=ax2)
ax2.set_title("SubSample Correlation Matrix", fontsize=14)
plt.show()

Unsupervised

In [ ]:
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD

In [ ]:
new_df = new_df.drop(columns=["P1_netPI"])

In [ ]:
new_df

In [ ]:
# New_df is from the random undersample data (fewer instances)
X = new_df.drop('label', axis=1)
y = new_df['label']

# T-SNE Implementation
t0 = time.time()
X_reduced_tsne = TSNE(n_components=2, random_state=42).fit_transform(X.values)
t1 = time.time()
print(f"T-SNE took {t1 - t0:.2} s")

# PCA Implementation
t0 = time.time()
X_reduced_pca = PCA(n_components=2, random_state=42).fit_transform(X.values)
t1 = time.time()
print(f"PCA took {t1 - t0:.2} s")

# TruncatedSVD
t0 = time.time()
X_reduced_svd = TruncatedSVD(n_components=2, algorithm='randomized', random_state=42).fit_transform(X.values)
t1 = time.time()
print(f"Truncated SVD took {t1 - t0:.2} s")

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24,6))
f.suptitle('Clusters using Dimensionality Reduction', fontsize=14)

blue_patch = mpatches.Patch(color='#0A0AFF', label='No bloom')
red_patch = mpatches.Patch(color='#AF0000', label='bloom')

# t-SNE scatter plot
ax1.scatter(X_reduced_tsne[:,0], X_reduced_tsne[:,1], c=(y == 0), cmap='coolwarm', label='No bloom', linewidths=2)
ax1.scatter(X_reduced_tsne[:,0], X_reduced_tsne[:,1], c=(y == 1), cmap='coolwarm', label='bloom', linewidths=2)
ax1.set_title('t-SNE', fontsize=14)

ax1.grid(True)

ax1.legend(handles=[blue_patch, red_patch])

# PCA scatter plot
ax2.scatter(X_reduced_pca[:,0], X_reduced_pca[:,1], c=(y == 0), cmap='coolwarm', label='No bloom', linewidths=2)
ax2.scatter(X_reduced_pca[:,0], X_reduced_pca[:,1], c=(y == 1), cmap='coolwarm', label='bloom', linewidths=2)
ax2.set_title('PCA', fontsize=14)

ax2.grid(True)

ax2.legend(handles=[blue_patch, red_patch])

# TruncatedSVD scatter plot
ax3.scatter(X_reduced_svd[:,0], X_reduced_svd[:,1], c=(y == 0), cmap='coolwarm', label='No bloom', linewidths=2)
ax3.scatter(X_reduced_svd[:,0], X_reduced_svd[:,1], c=(y == 1), cmap='coolwarm', label='bloom', linewidths=2)
ax3.set_title('Truncated SVD', fontsize=14)

ax3.grid(True)

ax3.legend(handles=[blue_patch, red_patch])

plt.show()

Classification

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix

In [ ]:
X = df_input.drop(columns=['P1_netPI', 'label'])
y = df_input['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

In [ ]:
for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score = cross_val_score(classifier, X_test, y_test, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")

In [ ]:
log_reg_sm = LogisticRegression()

In [ ]:
log_reg_sm.fit(X_train, y_train)

In [ ]:
y_pred_log_reg = log_reg_sm.predict(X_test)

In [ ]:
labels = ['No bloom', 'bloom']
plot_confusion_matrix(confusion_matrix(y_test, y_pred_log_reg), labels, title="Confusion Matrix", cmap=plt.cm.Reds)